
# Analysis Notebook

This notebook can be used to calculate statistical metrics for the data produced. 
*** 

## 0. Setup <a class="anchor" id="0"></a>
This section imports all files and sets up the notebook.

In [101]:
# Import all pacakges
import os
import pandas as pd
import numpy as np
from scipy import stats


In [102]:
# Choose which results to use
results_to_use = ["Random", "E_Greedy_Exp_Decay"]  # If this is empty, all files in the folder will be used
WARMUP_EPOCHS = 0 # Number of epochs to ignore.
sim_colours = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


all_results = os.listdir('results/')

if results_to_use == [""]:
    results_to_use = all_results
# remove DS_Store from the list
if '.DS_Store' in results_to_use:
    results_to_use.remove('.DS_Store')
    
print("Using results: " + str(results_to_use))


Using results: ['Random', 'E_Greedy_Exp_Decay']


In [103]:
# Create a dictionary. Each key is a set of results. 

all_sim_results = {}

for sim in results_to_use:
    all_sim_results[sim] = {}
    all_sim_results[sim]['configuration'] = pd.read_csv(
        'results/' + sim + '/configuration.txt', header=None, delimiter=":")
    types_of_results = []
    # Get all the filenames in the folder, excluding the extension
    for file in os.listdir('results/' + sim + '/exported_data/'):
        if file.endswith(".npy"):
            types_of_results.append(file[:-4])
    # Create a dictionary for each type of result
    for result in types_of_results:
        all_sim_results[sim][result] = np.load(
            'results/' + sim + '/exported_data/' + result + '.npy')

In [104]:
# Show all the all_sim_results in the notebook
for set_of_results in results_to_use:
    print("Results: " + set_of_results)
    print(all_sim_results[set_of_results]['configuration'])

Results: Random
                                    0                   1
0                             command                 run
1                       num_of_epochs               10000
2                  num_of_simulations                  10
3                           grid_size                   3
4                      queue_capacity                  10
5                     congestion_rate                0.07
6                       with_hotspots                True
7                           wage_time                  10
8                      credit_balance                   5
9           shared_auction_parameters                None
10  adaptive_auction_action_selection              random
11               bid_calculation_rule              linear
12       adaptive_auction_update_rule       simple_bandit
13             auction_episode_length                  10
14   action_selection_hyperparameters                None
15    adaptive_auction_discretization                  2

## 1. Average Total Number of Trips  <a class="anchor" id="1"></a>

This t-test compares the average total number of trips between the two experiments, for all sims.

In [105]:
all_average_number_of_trips = []
for sim in results_to_use:
    all_average_number_of_trips.append(
        all_sim_results[sim]['stat_num_of_trips_per_simulation'])

results = stats.ttest_ind(all_average_number_of_trips[0], all_average_number_of_trips[1])
print(results)
#present the results nicely
print("The average number of trips for the first two simulations are: " + str(np.mean(all_average_number_of_trips[0])) + " and " + str(np.mean(all_average_number_of_trips[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-2.3839727470307106, pvalue=0.02834310244952404, df=18.0)
The average number of trips for the first two simulations are: 24763.7 and 24883.8
The p-value is: 0.02834310244952404
The difference is significant


## 2. Average Congestion of central Intersection

This t-test is between the 2 experiments, for all sims. It does not process individual epochs

In [106]:
average_congestion_per_intersection = []
for exp in results_to_use:
    average_congestion_per_intersection.append(
        all_sim_results[exp]['stat_average_congestion_per_intersection'])

results = stats.ttest_ind(average_congestion_per_intersection[0][:,1,1], average_congestion_per_intersection[1][:,1,1])
print(results)
print("The average congestion for the first two simulations are: " + str(np.mean(average_congestion_per_intersection[0][:,1,1])) + " and " + str(np.mean(average_congestion_per_intersection[1][:,1,1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")

TtestResult(statistic=5.831636558031212, pvalue=1.5939058704940788e-05, df=18.0)
The average congestion for the first two simulations are: 0.32815325 and 0.31273275
The p-value is: 1.5939058704940788e-05
The difference is significant


## 3. Average Time Waited <a id="3"></a>

agent, intersection and grid based

In [107]:
######Agent based######
# Average time
average_time_agent = []
for exp in results_to_use:
    average_time_agent.append(
        all_sim_results[exp]['stat_average_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(average_time_agent[0], average_time_agent[1])
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(average_time_agent[0])) + " and " + str(np.mean(average_time_agent[1])))
print("The p-value is: " + str(results[1]))

# Max time
max_time_agent = []
for exp in results_to_use:
    max_time_agent.append(
        all_sim_results[exp]['stat_max_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(max_time_agent[0], max_time_agent[1])
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(max_time_agent[0])) + " and " + str(np.mean(max_time_agent[1])))
print("The p-value is: " + str(results[1]))

TtestResult(statistic=20.30700598460725, pvalue=7.403600266153952e-14, df=18.0)
The average time waited for the first two simulations are: 0.2757655555555555 and 0.2379477777777778
The p-value is: 7.403600266153952e-14
TtestResult(statistic=18.338219578216414, pvalue=4.282144123588275e-13, df=18.0)
The max time waited for the first two simulations are: 0.7119411111111111 and 0.6166555555555555
The p-value is: 4.282144123588275e-13


In [108]:
######Intersection based - Average Time######

average_time_intersection = []
for exp in results_to_use:
    average_time_intersection.append(
        all_sim_results[exp]['stat_average_time_waited_per_intersection'])
    
for i in range(len(average_time_intersection[0][0])):
    for j in range(len(average_time_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_time_intersection[0][:,i,j], average_time_intersection[1][:,i,j])
        print(results)
        print("The average time waited for the first two simulations are: " + str(np.mean(average_time_intersection[0][:,i,j])) + " and " + str(np.mean(average_time_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=-1.8347096462753625, pvalue=0.0831318127408832, df=18.0)
The average time waited for the first two simulations are: 0.0231125 and 0.026392500000000003
The p-value is: 0.0831318127408832
Intersection [ 0 ][ 1 ]
TtestResult(statistic=1.3871492403803547, pvalue=0.182334090048716, df=18.0)
The average time waited for the first two simulations are: 0.24544499999999997 and 0.23470249999999998
The p-value is: 0.182334090048716
Intersection [ 0 ][ 2 ]
TtestResult(statistic=-1.0479265580533594, pvalue=0.30854047577592786, df=18.0)
The average time waited for the first two simulations are: 0.023865 and 0.025095
The p-value is: 0.30854047577592786
Intersection [ 1 ][ 0 ]
TtestResult(statistic=-1.6219612271316932, pvalue=0.12219610969865431, df=18.0)
The average time waited for the first two simulations are: 0.3023175 and 0.32062749999999995
The p-value is: 0.12219610969865431
Intersection [ 1 ][ 1 ]
TtestResult(statistic=13.074391167469198, pvalue=1.2

In [109]:
######Intersection based - Max Time######

max_time_intersection = []
for exp in results_to_use:
    max_time_intersection.append(
        all_sim_results[exp]['stat_max_time_waited_per_intersection'])
    
for i in range(len(max_time_intersection[0][0])):
    for j in range(len(max_time_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(max_time_intersection[0][:,i,j], max_time_intersection[1][:,i,j])
        print(results)
        print("The average time waited for the first two simulations are: " + str(np.mean(max_time_intersection[0][:,i,j])) + " and " + str(np.mean(max_time_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=-1.9938529068505888, pvalue=0.06154416981666345, df=18.0)
The average time waited for the first two simulations are: 0.08226000000000001 and 0.09254000000000001
The p-value is: 0.06154416981666345
Intersection [ 0 ][ 1 ]
TtestResult(statistic=1.4936859508034832, pvalue=0.15258049595304954, df=18.0)
The average time waited for the first two simulations are: 0.6872 and 0.6571400000000001
The p-value is: 0.15258049595304954
Intersection [ 0 ][ 2 ]
TtestResult(statistic=-1.1259707556256784, pvalue=0.2749669076280165, df=18.0)
The average time waited for the first two simulations are: 0.08464999999999999 and 0.08861
The p-value is: 0.2749669076280165
Intersection [ 1 ][ 0 ]
TtestResult(statistic=-1.3981283459616458, pvalue=0.17906598852197048, df=18.0)
The average time waited for the first two simulations are: 0.82746 and 0.8667400000000001
The p-value is: 0.17906598852197048
Intersection [ 1 ][ 1 ]
TtestResult(statistic=12.702446384019048, pval

In [110]:
######Grid based######

# Average time
print("AVERAGE TIME WAITED")
average_time_grid = []
for exp in results_to_use:
    average_time_grid.append(
        all_sim_results[exp]['stat_average_time_waited_grid'])

means_1 = []
for i in range(len(average_time_grid[0])):
    means_1.append(np.mean(average_time_grid[0][i]))
means_2 = []
for i in range(len(average_time_grid[1])):
    means_2.append(np.mean(average_time_grid[1][i]))


results = stats.ttest_ind(means_1, means_2)
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))


# Max time
print("MAX TIME WAITED")
max_time_grid = []
for exp in results_to_use:
    max_time_grid.append(
        all_sim_results[exp]['stat_max_time_waited_grid'])

means_1 = []
for i in range(len(max_time_grid[0])):
    means_1.append(np.mean(max_time_grid[0][i]))
means_2 = []
for i in range(len(max_time_grid[1])):
    means_2.append(np.mean(max_time_grid[1][i]))


results = stats.ttest_ind(means_1, means_2)
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))

AVERAGE TIME WAITED
TtestResult(statistic=20.30700598460726, pvalue=7.403600266153899e-14, df=18.0)
The average time waited for the first two simulations are: 0.2757655555555556 and 0.2379477777777778
The p-value is: 7.403600266153899e-14
MAX TIME WAITED
TtestResult(statistic=18.338219578216368, pvalue=4.2821441235884436e-13, df=18.0)
The max time waited for the first two simulations are: 0.7119411111111111 and 0.6166555555555556
The p-value is: 4.2821441235884436e-13


## 4. Gini coefficient <a id="4"></a>

In [111]:
# Average Time Waited.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_time_waited = []
for sim in results_to_use:
    all_ginis_time_waited.append(
        all_sim_results[sim]['stat_time_waited_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of time waited for the first two simulations are: " + str(np.mean(all_ginis_time_waited[0])) + " and " + str(np.mean(all_ginis_time_waited[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-5.385768304026473, pvalue=4.0596978338806996e-05, df=18.0)
The average gini of time waited for the first two simulations are: 0.1342073960076311 and 0.14004576193543522
The p-value is: 4.0596978338806996e-05
The difference is significant


In [112]:
# Average Satisfaction.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_satisfaction = []
for sim in results_to_use:
    all_ginis_satisfaction.append(
        all_sim_results[sim]['stat_satisfaction_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of satisfaction for the first two simulations are: " + str(np.mean(all_ginis_satisfaction[0])) + " and " + str(np.mean(all_ginis_satisfaction[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-5.385768304026473, pvalue=4.0596978338806996e-05, df=18.0)
The average gini of satisfaction for the first two simulations are: 0.43450183023533107 and 0.4267730882164245
The p-value is: 4.0596978338806996e-05
The difference is significant


## 5. Trip Satisfaction <a id="5"></a>

In [113]:
all_satisfactions = []
for sim in results_to_use:
    all_satisfactions.append(
        all_sim_results[sim]['stat_satisfaction_mean'])

results = stats.ttest_ind(all_satisfactions[0], all_satisfactions[1])
print(results)
#present the results nicely
print("The average satisfactino for the first two simulations are: " + str(np.mean(all_satisfactions[0])) + " and " + str(np.mean(all_satisfactions[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-4.831750046471582, pvalue=0.0001338026967753625, df=18.0)
The average satisfactino for the first two simulations are: 0.15497821877692497 and 0.1566294076515055
The p-value is: 0.0001338026967753625
The difference is significant


## 6. Auction Reward <a id="5"></a>

In [114]:
######Intersection based - Average Auction Reward######

average_reward_intersection = []
for exp in results_to_use:
    average_reward_intersection.append(
        all_sim_results[exp]['stat_average_auction_reward_per_intersection'])
    
for i in range(len(average_reward_intersection[0][0])):
    for j in range(len(average_reward_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_reward_intersection[0][:,i,j], average_reward_intersection[1][:,i,j])
        print(results)
        print("The average auction reward for the first two simulations are: " + str(np.mean(average_reward_intersection[0][:,i,j])) + " and " + str(np.mean(average_reward_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=-1.2878762175796052, pvalue=0.2141007031489009, df=18.0)
The average auction reward for the first two simulations are: 0.04920416666666667 and 0.051151666666666665
The p-value is: 0.2141007031489009
Intersection [ 0 ][ 1 ]
TtestResult(statistic=-1.7340523802142798, pvalue=0.10000203606109737, df=18.0)
The average auction reward for the first two simulations are: 0.2450558333333333 and 0.2503558333333333
The p-value is: 0.10000203606109737
Intersection [ 0 ][ 2 ]
TtestResult(statistic=-1.4250817349180562, pvalue=0.17124317379775145, df=18.0)
The average auction reward for the first two simulations are: 0.04953416666666667 and 0.05115583333333333
The p-value is: 0.17124317379775145
Intersection [ 1 ][ 0 ]
TtestResult(statistic=-3.48126312371585, pvalue=0.0026653144364041878, df=18.0)
The average auction reward for the first two simulations are: 0.24761666666666665 and 0.26056749999999995
The p-value is: 0.0026653144364041878
Intersection [ 1 